# HW3 Code

In [40]:
import numpy as np
import jax, jax.numpy as jnp

In [41]:
# Define Cart-pole constants
mp = 2.0  # pendulum mass
mc = 10.0  # cart mass
L = 1.0  # pendulum length
g = 9.81  # gravitational acceleration
dt = 0.1  # discretization time step

Q = jnp.eye(4)
R = jnp.eye(1)

In [42]:
df_ds = jnp.array(
    [
        [0, 0, 1, 0],
        [0, 0 ,0, 1],
        [0, mp*g/mc, 0, 0],
        [0, (mc+mp)*g/(mc*L), 0, 0]
    ]
)

df_du = jnp.array(
    [
        [0],
        [0],
        [1/mc],
        [1/(mc*L)]
    ]
)

In [46]:
A = jnp.eye(4) + dt * df_ds

B = dt * df_du
B

Array([[0.  ],
       [0.  ],
       [0.01],
       [0.01]], dtype=float32)

In [47]:
# Final cost-to-go P_N
P_N = 0*jnp.eye(4)

def riccati(P_k1, A, B, eps=1e-4):

    K_k = -1 * jnp.linalg.inv(R + B.T @ P_k1 @ B) @ B.T @ P_k1 @ A

    P_k = Q + A.T @ P_k1 @ (A + B @ K_k)

    # print(K_k.T @ R @ K_k)

    # P_k = Q + K_k.T @ R @ K_k + (A + B @ K_k).T @ P_k1 @ (A + B @ K_k)

    # termination condition
    if jnp.max(jnp.abs(P_k1-P_k)) < 1e-4:
        print('Convergence achieved')

    else:
        K_k, P_k = riccati(P_k, A, B)

    return K_k, P_k

In [48]:
K_inf, P_inf = riccati(P_N, A, B, eps=1e-4)
print("K_inf:\n", K_inf)
print("P_inf:\n", P_inf)

Convergence achieved
K_inf:
 [[   0.7291392 -231.85417      4.219666   -68.24741  ]]
P_inf:
 [[ 5.7871891e+01 -9.3599890e+02  1.5956427e+02 -2.9671216e+02]
 [-9.3599847e+02  1.2596546e+05 -5.0264883e+03  3.7501445e+04]
 [ 1.5956425e+02 -5.0264912e+03  8.1205951e+02 -1.5920457e+03]
 [-2.9671207e+02  3.7501449e+04 -1.5920449e+03  1.1181582e+04]]
